In [47]:
import tensorflow as tf
import coremltools as ct
import keras
from tensorflow.keras import utils as np_utils
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D 
from keras.layers import Dense, Dropout, Flatten
# 定数の定義
img_rows, img_cols = 28, 28 
input_shape = (img_rows, img_cols, 1) 
num_classes = 10

# モデルの構築
model = Sequential() 
model.add(Conv2D(32, kernel_size=(3, 3), 
                 activation='relu', 
                 input_shape=input_shape))

model.add(Conv2D(64, (3, 3), activation='relu')) 
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu')) 
model.add(Dropout(0.5)) 
model.add(Dense(num_classes, activation='softmax'))

In [33]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 128)               1179776   
_________________________________________________________________
dropout_11 (Dropout)         (None, 128)              

In [34]:
 model.compile(loss=keras.losses.categorical_crossentropy,
               optimizer=keras.optimizers.gradient_descent_v2.SGD(),
               metrics=['accuracy'])

In [36]:
# MNISTデータをロード
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [37]:
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1) 
x_train = x_train.astype('float32')
x_train /= 255
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1) 
x_test = x_test.astype('float32')
x_test /= 255

In [38]:
y_train = np_utils.to_categorical(y_train, num_classes) 
y_test = np_utils.to_categorical(y_test, num_classes)

In [39]:
print('x_train.shape:', x_train.shape) 
print('x_test.shape:', x_test.shape) 
print('y_train.shape:', y_train.shape) 
print('y_test.shape:', y_test.shape)

x_train.shape: (60000, 28, 28, 1)
x_test.shape: (10000, 28, 28, 1)
y_train.shape: (60000, 10)
y_test.shape: (10000, 10)


In [40]:
model.fit(x_train, 
          y_train, 
          batch_size=128,
          epochs=12,
          verbose=1)

Epoch 1/12
469/469 [==============================] - 46s 96ms/step - loss: 1.0124 - accuracy: 0.6816
Epoch 2/12
469/469 [==============================] - 45s 95ms/step - loss: 0.4435 - accuracy: 0.8642
Epoch 3/12
469/469 [==============================] - 46s 98ms/step - loss: 0.3648 - accuracy: 0.8900
Epoch 4/12
469/469 [==============================] - 45s 97ms/step - loss: 0.3248 - accuracy: 0.9039
Epoch 5/12
469/469 [==============================] - 45s 95ms/step - loss: 0.2962 - accuracy: 0.9111
Epoch 6/12
469/469 [==============================] - 45s 95ms/step - loss: 0.2760 - accuracy: 0.9170
Epoch 7/12
469/469 [==============================] - 45s 96ms/step - loss: 0.2612 - accuracy: 0.9220
Epoch 8/12
469/469 [==============================] - 44s 93ms/step - loss: 0.2517 - accuracy: 0.9234
Epoch 9/12
469/469 [==============================] - 44s 94ms/step - loss: 0.2352 - accuracy: 0.9296
Epoch 10/12
469/469 [==============================] - 44s 95ms/step - loss: 0.224

In [41]:
score = model.evaluate(x_test, y_test, verbose=0)

In [42]:
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.10686808079481125
Test accuracy: 0.9671000242233276


In [43]:
model.save('./KerasMnist.h5')

In [44]:
from keras.models import load_model
keras_model = load_model('./KerasMnist.h5')

In [60]:
# mlmodel = ct.convert(keras_model,
#                     inputs=[ct.TensorType(name="image")],
#                     outputs=[ct.TensorType(name="digitProbabilities")])
class_labels = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
mlmodel = ct.convert(keras_model,
                     source='tensorflow',
#                      inputs=[ct.TensorType(name='image')],
#                      outputs=[ct.TensorType(name='digitProbabilities')],
                     classifier_config=ct.ClassifierConfig(class_labels,
                                                         predicted_feature_name='digit'))

2022-07-02 10:42:02.605659: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2022-07-02 10:42:02.605742: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2022-07-02 10:42:02.613143: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1137] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.004ms.
  function_optimizer: function_optimizer did nothing. time = 0.001ms.

2022-07-02 10:42:02.758491: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2022-07-02 10:42:02.758596: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2022-07-02 10:42:02.838546: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:113

In [61]:
mlmodel.save('./MNISTDigitClassifier.mlmodel')

In [63]:
spec = ct.utils.load_spec('./MNISTDigitClassifier.mlmodel')
builder = ct.models.neural_network.NeuralNetworkBuilder(spec=spec)
builder.inspect_input_features()



[Id: 0] Name: conv2d_10_input
          Type: multiArrayType {
  shape: 1
  shape: 28
  shape: 28
  shape: 1
  dataType: FLOAT32
  shapeRange {
    sizeRanges {
      lowerBound: 1
      upperBound: -1
    }
    sizeRanges {
      lowerBound: 28
      upperBound: 28
    }
    sizeRanges {
      lowerBound: 28
      upperBound: 28
    }
    sizeRanges {
      lowerBound: 1
      upperBound: 1
    }
  }
}



In [64]:
from coremltools.proto import FeatureTypes_pb2 as ft
grayscale = ft.ImageFeatureType.ColorSpace.Value('GRAYSCALE') 
input_image_type = builder.spec.description.input[0].type.imageType 
input_image_type.width = 28
input_image_type.height = 28
input_image_type.colorSpace = grayscale

builder.inspect_input_features()

[Id: 0] Name: conv2d_10_input
          Type: imageType {
  width: 28
  height: 28
  colorSpace: GRAYSCALE
}



In [66]:
mlmodel_modified = ct.models.MLModel(spec) 
mlmodel_modified.save('./ModifiedMNISTDigitClassifier.mlmodel')